El problema de Monty Hall
(revisitado)

* El concursante tiene una oportunidad de
cambiar su elección después de haber visto
MH:
    * Si decide cambiar entonces X=2 y si no
decide cambiar entonces X=0  
* Si al finalizar el juego el concursante gana
entonces Y = 1, de otra forma Y = -1

Z= Elección  
W = Ganador  
MH = Apertura de puerta  


En presencia de Monty Hall:

$$
\mathbb{P}(W \mid Z, MH) \neq \mathbb{P}(W \mid MH)
$$

In [4]:
import pandas as pd

In [28]:
data = [
    ['A', 'A', 'B',  2, -1],
    ['A', 'A', 'C',  2, -1],
    ['A', 'B', 'C',  2,  1],
    ['A', 'B', 'C',  2,  1],
    ['A', 'C', 'B',  2,  1],
    ['A', 'C', 'B',  2,  1],
    ['B', 'A', 'C',  2,  1],
    ['B', 'A', 'C',  2,  1],
    ['B', 'B', 'A',  2, -1],
    ['B', 'B', 'C',  2, -1],
    ['B', 'C', 'A',  2,  1],
    ['B', 'C', 'A',  2,  1],
    ['C', 'A', 'B',  2,  1],
    ['C', 'A', 'B',  2,  1],
    ['C', 'B', 'A',  2,  1],
    ['C', 'B', 'A',  2,  1],
    ['C', 'C', 'A',  2, -1],
    ['C', 'C', 'B',  2, -1],
    ['A', 'A', 'B',  0,  1],
    ['A', 'A', 'C',  0,  1],
    ['A', 'B', 'C',  0, -1],
    ['A', 'B', 'C',  0, -1],
    ['A', 'C', 'B',  0, -1],
    ['A', 'C', 'B',  0, -1],
    ['B', 'A', 'C',  0, -1],
    ['B', 'A', 'C',  0, -1],
    ['B', 'B', 'A',  0,  1],
    ['B', 'B', 'C',  0,  1],
    ['B', 'C', 'A',  0, -1],
    ['B', 'C', 'A',  0, -1],
    ['C', 'A', 'B',  0, -1],
    ['C', 'A', 'B',  0, -1],
    ['C', 'B', 'A',  0, -1],
    ['C', 'B', 'A',  0, -1],
    ['C', 'C', 'A',  0,  1],
    ['C', 'C', 'B',  0,  1],
]

df = pd.DataFrame(data, columns=['Z', 'W', 'MH', 'X', 'Y'])


* Lo que deberíamos de constatar es que si la puerta inical seleccionada Z resulta ser igual a la ganadora W, lo cuál sucede 1/3 del tiempo, perdemos si cambiamos, pues ya estábamos ok. 
* EOC, Z <> W. Cambias nos hace ganar. Esto sucede 2/3 partes del tiempo, ya que es más fácil elegir una cabra que un auto, para después corregit.

Task: Llena la tabla con todos los casos y computa la probabildad de ganar con estrategia X=0 (mantener) vs. X=2 (cambiar).

In [27]:
def check_y(df):
    """Esta función se apoya en la observación de que si Z y W son iguales, uno no debería cambiar."""
    for idx, row in df.iterrows():
        if (row['Z'] == row['W']) and (row['X'] == 2):
            df.loc[idx, "y_check"] = -1
        elif (row['Z'] == row['W']) and (row['X'] == 0):
            df.loc[idx, "y_check"] = 1
        elif (row['Z'] != row['W']) and (row['X'] == 2):
            df.loc[idx, "y_check"] = 1
        elif (row['Z'] != row['W']) and (row['X'] == 0):
            df.loc[idx, "y_check"] = -1
    return df


In [30]:
df = check_y(df)  # Asegúrate que esta función asigna bien 'y_check'

# Calcula esperanza de Y
esperanza_y_x0 = df.loc[df['X'] == 0, 'Y'].mean()
esperanza_y_x2 = df.loc[df['X'] == 2, 'Y'].mean()

# Convierte esperanza a probabilidad de ganar (si Y es 1/-1)
prob_ganar_x0 = (esperanza_y_x0 + 1) / 2
prob_ganar_x2 = (esperanza_y_x2 + 1) / 2

print(f"Probabilidad ganar dado X=0 (mantener): {prob_ganar_x0:.3f}")
print(f"Probabilidad ganar dado X=2 (cambiar): {prob_ganar_x2:.3f}")

# Igual para y_check si quieres comparar
esperanza_ycheck_x0 = df.loc[df['X'] == 0, 'y_check'].mean()
esperanza_ycheck_x2 = df.loc[df['X'] == 2, 'y_check'].mean()

prob_ganar_ycheck_x0 = (esperanza_ycheck_x0 + 1) / 2
prob_ganar_ycheck_x2 = (esperanza_ycheck_x2 + 1) / 2

print(f"Probabilidad y_check ganar X=0: {prob_ganar_ycheck_x0:.3f}")
print(f"Probabilidad y_check ganar X=2: {prob_ganar_ycheck_x2:.3f}")


Probabilidad ganar dado X=0 (mantener): 0.333
Probabilidad ganar dado X=2 (cambiar): 0.667
Probabilidad y_check ganar X=0: 0.333
Probabilidad y_check ganar X=2: 0.667


In [29]:
print(df.groupby('X')['Y'].value_counts(normalize=True))

X  Y 
0  -1    0.666667
    1    0.333333
2   1    0.666667
   -1    0.333333
Name: proportion, dtype: float64


¿Entonces es lo mismo cambiar que no cambiar?
No es lo mismo. Sí importa porque ...Falta multiplicar por la probabilidad de que

## Explicación de por qué

$$
\text{prob\_ganar\_x0} = \frac{\mathbb{E}[Y \mid X=0] + 1}{2}
$$

---

### Contexto: codificación de la variable \(Y\)

En tu caso, la variable \(Y\) está codificada como:

- $Y = +1$ si **ganas**,
- $Y = -1$ si **pierdes**.

---

### ¿Qué significa $\mathbb{E}[Y \mid X=x]$?

La esperanza condicional es el valor promedio de \(Y\) dado que \(X = x\):

$$
\mathbb{E}[Y \mid X=x] = P(Y=1 \mid X=x) \cdot 1 + P(Y=-1 \mid X=x) \cdot (-1)
$$

Como $P(Y=1 \mid X=x) + P(Y=-1 \mid X=x) = 1$, podemos escribir:

$$
\mathbb{E}[Y \mid X=x] = P(Y=1 \mid X=x) - P(Y=-1 \mid X=x)
$$

---

### Despejando la probabilidad de ganar

Queremos encontrar $P(Y=1 \mid X=x)$, que es la probabilidad de ganar dado $X=x$.

De la ecuación anterior:

$$
\mathbb{E}[Y \mid X=x] = P(Y=1 \mid X=x) - (1 - P(Y=1 \mid X=x)) = 2 \cdot P(Y=1 \mid X=x) - 1
$$

Por lo tanto:

$$
P(Y=1 \mid X=x) = \frac{\mathbb{E}[Y \mid X=x] + 1}{2}
$$

---

### Intuición

- Si \($\mathbb{E}[Y \mid X=x] = 1$\), entonces ganas siempre, y \(P(Y=1 \mid X=x) = 1\).
- Si \($\mathbb{E}[Y \mid X=x] = -1$\), entonces pierdes siempre, y \(P(Y=1 \mid X=x) = 0\).
- Si \($\mathbb{E}[Y \mid X=x] = 0$\), ganas y pierdes con igual probabilidad, \(P(Y=1 \mid X=x) = 0.5\).

---

### Resumen

| $\mathbb{E}[Y \mid X=x]$| $P(Y=1 \mid X=x)$ | Interpretación                |
|----------------------------|---------------------|------------------------------|
| 1                          | 1                   | Ganas siempre                |
| 0                          | 0.5                 | Ganas la mitad de las veces  |
| -1                         | 0                   | Nunca ganas                  |

---
